In [0]:
pip install geopandas

In [0]:
%python

import pandas as pd
from datetime import datetime
import geopandas as gpd
from shapely import wkt, union_all, intersection


In [0]:
pigeons = pd.read_csv('data/pigeonnier_update.csv',sep=';',skiprows=1)
pigeons.columns = ['date','pedestrians','debug','dropped']
pigeons.date = pigeons['date'].apply(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S").date())

In [0]:
%python

pigeons[pigeons.pedestrians>0][['date','pedestrians']].head(2)

# Outdoor Vision : Outings

In [0]:
ov_file = "data/CC_Gorges_Ardeche_2020_2024_0.gpkg"
outdoor_vision = gpd.read_file(ov_file)

In [0]:
#outdoor_vision.provider.drop_duplicates()
#outdoor_vision.activity.drop_duplicates()
red_outdoor = outdoor_vision[['activity','user_id','date_start','date_end','geometry']]
red_outdoor.head(1)

In [0]:
red_outdoor.shape

# Biodiv : Sensitive Areas

In [0]:
biodiv = gpd.read_file("https://object.files.data.gouv.fr/dataeng-open/hackathon_outdoor/SIG/03-Biodiversite/Data_biodiv/data_geonatureSGGA.geojson")

In [0]:
biodiv.head(1)

In [0]:
biodiv.columns

In [0]:
biodiv['buffer_test'] = biodiv['geometry'].buffer(1)

to_keep = ['biogeographique_statut', 'biologique_statut','cd_habitat', 'cd_nom','classe', 'comment_occurrence','comportement',
       'date_creation', 'date_debut', 'date_fin', 'date_modification',
       'date_validation', 'etat_biologique', 'famille','group1_inpn', 'group2_inpn',
       'group3_inpn', 
       'naturalite', 
       'niveau_sensibilite', 'nom_cite', 'nom_vernaculaire', 'nombre_max',
       'nombre_min', 'objet_denombrement',  'ordre', 'rang_taxo',
       'regne','stade_vie', 'statut_observation',
       'statut_source', 'technique_observation', 'type_denombrement',
       'type_info_geo', 'type_regroupement',  'x_centroid_4326', 'y_centroid_4326',
       'geometry', 'geometrie_wkt_4326', 'buffer_test']
to_keep_min = ['nom_cite','geometry', 'geometrie_wkt_4326', 'buffer_test']

In [0]:
red_biodiv = biodiv[to_keep_min]
red_biodiv.head(1)

In [0]:
#red_biodiv = gpd.GeoDataFrame(red_biodiv,geometry=red_biodiv['buffer_test'].apply(wkt.loads))
df_agg_bs = red_biodiv.groupby(['nom_cite']).agg({'buffer_test':lambda x: list(x)}).reset_index()

#df_agg_bs = gpd.GeoSeries(lambda x : x.buffer_test)

df_agg_bs.head(3)

In [0]:
#from shapely.ops import unary_union

#df_agg_bs = df_agg_bs.groupby(['nom_cite']).apply(lambda x: unary_union(list(x.buffer_test))).reset_index(name='full_poly')

In [0]:
df_agg_bs.head(1)

In [0]:

# turn all areas (geometries) into one big area of same criteria
df_agg_bs = df_agg_bs.groupby(['nom_cite']).apply(lambda x: union_all(list(x.buffer_test)))\
                     .reset_index(name='geometry')

In [0]:
df_agg_bs.head(1)

# Intersects

In [0]:
in_bs = df_agg_bs.join(red_outdoor.head(50), lsuffix='_bs', rsuffix='_otd', how='cross')
in_bs['intersects'] = in_bs.apply(lambda x: x.geometry_bs.intersects(x.geometry_otd), axis=1)
in_bs = in_bs[in_bs['intersects']==True]#[['nom_cite','activity','user_id','date_start','date_end',]]

In [0]:
in_bs.head(1)

In [0]:
import matplotlib.pyplot as plt

df_plot = gpd.GeoDataFrame()
df_plot['geometry'] = [df_all_agg[df_all_agg['otd_exp_id'] == '62815f0535238'].reset_index()[['geometry_bs']].iloc[0, 0],
                       df_all_agg[df_all_agg['otd_exp_id'] == '62815f0535238'].reset_index()[['geometry_otd']].iloc[0, 0]]
df_plot['color'] = ['green', 'red']
df_plot = df_plot.set_geometry('geometry')

df_plot.plot(figsize=(8, 8), color=df_plot['color'])
plt.show()